# Loading Data From nba_api And Creating a CSV File With The Data

In [ ]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import pandas as pd
import concurrent.futures
import numpy as np

In [ ]:
### Importing Player Names/IDs. List of dictionaries
data = players.get_players()
data

In [ ]:
### Creating a list of Player IDs
player_ids = [player['id'] for player in data]
player_ids

In [ ]:
### Fetches Player Career Stats using Player IDs
def player_career_stats(player_id):
    try:
        career = playercareerstats.PlayerCareerStats(player_id = player_id)
        return career.get_data_frames()[1]
    except Exception as e:
        print(f"Error retrieving stats for player ID {player_id}: {e}")
        return pd.DataFrame()

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(player_career_stats, player_ids))

# Concatenate results
career_totals = pd.concat(results, ignore_index=True)

In [ ]:
### Sorting the Data Frame by Player_IDs
career_totals = career_totals.sort_values('PLAYER_ID')
career_totals

In [ ]:
### Creating list of Player_IDs and Player_Names
player_names = [(player['id'], player['full_name']) for player in data]
player_names = pd.DataFrame(player_names, columns=['PLAYER_ID', 'PLAYER_NAME'])
player_names

In [ ]:
### Joining player_names with career_totals to add PLAYER_NAMEs to the Data Frame. Joining on PLAYER_IDs 
career_totals = pd.merge(career_totals, player_names, on = 'PLAYER_ID', how = 'outer')
career_totals

In [ ]:
### Moving PLAYER_NAME to the second column of the Data Frame to read easier
column_to_move = career_totals.pop('PLAYER_NAME')
career_totals.insert(1, 'PLAYER_NAME', column_to_move)
career_totals

In [ ]:
### Removing unecessary columns
career_totals = career_totals.drop('LEAGUE_ID', axis=1)
career_totals = career_totals.drop('Team_ID', axis=1)

In [ ]:
### Creating CSV File
career_totals.to_csv('playercareertotals.csv', index=False)